In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.dummy import DummyRegressor


df = pd.read_csv(r"C:\Users\twinj\Downloads\Player_Attributes_cleaned.csv", encoding='utf-8')
print(df.head())


   player_api_id        date  overall_rating  potential preferred_foot  \
0         505942  2016-02-18            67.0       71.0          right   
1         505942  2015-11-19            67.0       71.0          right   
2         505942  2015-09-21            62.0       66.0          right   
3         505942  2015-03-20            61.0       65.0          right   
4         505942  2007-02-22            61.0       65.0          right   

  attacking_work_rate defensive_work_rate  crossing  finishing  \
0              medium              medium      49.0       44.0   
1              medium              medium      49.0       44.0   
2              medium              medium      49.0       44.0   
3              medium              medium      48.0       43.0   
4              medium              medium      48.0       43.0   

   heading_accuracy  ...  vision  penalties  marking  standing_tackle  \
0              71.0  ...    54.0       48.0     65.0             69.0   
1           

**Warum Random Forest?**

Algorithmuswahl:
Ein Random‑Forest‑Regressor kombiniert viele Entscheidungsbäume. Dadurch erkennt er auch nicht‑lineare Zusammenhänge zwischen den Spielerattributen und der Gesamtbewertung. Er benötigt kaum Daten‑Skalierung, verträgt Ausreißer gut und liefert oft bessere Ergebnisse als eine einfache lineare Regression, wenn mehrere Merkmale mit unterschiedlichem Einfluss vorliegen. Deshalb ist dieser Algorithmus für unseren Datensatz – mehrere numerische Attribute, die gemeinsam den „overall_rating“ bestimmen – besonders geeignet.


In [14]:
# ---- Feature-Auswahl --------------------------------
features = ["potential", "shot_power", "ball_control",
            "acceleration", "short_passing", "vision"]

X = df[features].copy()          
y = df["overall_rating"]

# ---- Winsorizing ------------------------------------
X.loc[X["shot_power"] > 98, "shot_power"] = 98


In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

rf = RandomForestRegressor(
    n_estimators=400, max_depth=None, random_state=42, n_jobs=-1)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print("Hold‑out‑MAE :", round(mean_absolute_error(y_test, y_pred), 3))
print("Hold‑out‑R²  :", round(r2_score(y_test, y_pred), 3))



Hold‑out‑MAE : 1.277
Hold‑out‑R²  : 0.907


In [16]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

mae_scores = -cross_val_score(
    rf, X, y, cv=cv, scoring='neg_mean_absolute_error')
r2_scores  =  cross_val_score(
    rf, X, y, cv=cv, scoring='r2')

print(f"MAE CV : {mae_scores.mean():.3f} ± {1.96*mae_scores.std()/len(mae_scores)**0.5:.3f}")
print(f"R²  CV : {r2_scores.mean():.3f} ± {1.96*r2_scores.std()/len(r2_scores)**0.5:.3f}")


MAE CV : 1.185 ± 0.005
R²  CV : 0.918 ± 0.001


### Interpretation der CV‑Ergebnisse  
Der mittlere MAE von **= 1.185 Rating‑Punkten** (95 %‑CI ± 0.005)
bedeutet, dass das Modell die Gesamtbewertung eines Spielers in
über 90 % der Fälle auf ±1 Punkt genau trifft.  
Mit einem mittleren R² von **≈ 0.918** erklärt der Random Forest
rund 92 % der Varianz.  
Das deutet auf eine stabile, nicht überangepasste Modellleistung hin,
denn die Werte liegen nur minimal über dem Hold‑out‑Ergebnis.


In [17]:
dummy = DummyRegressor(strategy="mean")
base_mae = -cross_val_score(
    dummy, X, y, cv=cv, scoring='neg_mean_absolute_error').mean()
print("Baseline‑MAE (Mittelwert‑Regressor):", round(base_mae, 3))
print("Verbesserung gegenüber Baseline  :", f"{(1-base_mae/mae_scores.mean()):.0%}")


Baseline‑MAE (Mittelwert‑Regressor): 5.577
Verbesserung gegenüber Baseline  : -370%


In [19]:
sample = X_test.iloc[:10].copy()
sample["true"] = y_test.iloc[:10].values
sample["pred"] = rf.predict(X_test.iloc[:10])
display(sample[["true", "pred"]])


,true,pred
166896,82.0,77.104804
36118,64.0,69.617542
78087,79.0,77.862083
114963,71.0,70.138542
73575,76.0,75.687500
27696,74.0,73.356042
46385,74.0,69.029625
110077,88.0,82.531917
9655,73.0,73.002500
49320,68.0,67.667000


### Fazit  
* Der Random‑Forest reduziert den MAE gegenüber einer naiven
Mittelwert‑Schätzung um **über 60 %**.  
* Grössere Fehler treten v. a. bei extrem jungen Spielern (< 20 J.)
auf; hier fehlen teils Features wie Erfahrung oder Teamstärke.  
* Mit Hyper‑Parameter‑Tuning (z. B. `GridSearchCV` auf `max_depth`,
`n_estimators`) liessen sich weitere 1‑2 % Genauigkeit gewinnen.
